In [5]:
# ── 03_feature_engineering_refactored.ipynb: 윈도우 최적값 탐색 (jinja2-free) ──

import os
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm.auto import tqdm

# ── 1. 경로 & 데이터 로드
BASE      = "/Users/gun/Desktop/미래에셋 AI 공모전/data"
DIV_PATH  = os.path.join(BASE, "dividend_ml_ready.csv")
HIST_PATH = os.path.join(BASE, "price_history.csv")

df_div  = pd.read_csv(DIV_PATH, dtype={"stock_code":str})
df_div["rcept_dt"] = pd.to_datetime(
    df_div["rcept_no"].astype(str).str[:8],
    format="%Y%m%d", errors="coerce"
)
df_div = df_div.dropna(subset=["rcept_dt"]).reset_index(drop=True)

df_hist = pd.read_csv(HIST_PATH, parse_dates=["date"], dtype={"stock_code":str})

# ── 2. 원본 이벤트 수
orig_n = len(df_div)
print(f"▶ 총 이벤트 수: {orig_n:,}\n")

# ── 3. 윈도우 후보 정의
win_candidates = {
    "classification": [1, 2, 3],          # ±1~3 거래일
    "regression":     list(range(3, 11)), # ±3~10 거래일
    "clustering":     list(range(11, 22)) # ±11~21 거래일
}

# ── 4. 보존 개수 계산 함수
def win_len(g, dt, w):
    g = g.sort_values("date").reset_index(drop=True)
    pos = g["date"].searchsorted(dt)
    return len(g.iloc[max(pos-w,0): pos+w+1])

def count_preserved(window):
    kept = 0
    # df_hist에 이미 rcept_dt 컬럼이 있다고 가정
    for (_, dt), grp in df_hist.groupby(["stock_code","rcept_dt"]):
        if win_len(grp, dt, window) >= 2*window + 1:
            kept += 1
    return kept

# ── 5. 모듈별 윈도우 순회 및 결과 수집
results = []
for module, windows in win_candidates.items():
    for w in windows:
        kept = count_preserved(w)
        pct  = kept / orig_n
        results.append({
            "module": module,
            "window": w,
            "kept":   kept,
            "pct":    pct
        })

df_res = pd.DataFrame(results)
# 퍼센트 문자열 포맷
df_res["pct_str"] = (df_res["pct"]*100).round(1).astype(str) + "%"

# ── 6. 결과 출력
print("=== 윈도우별 보존 결과 ===")
print(df_res[["module","window","kept","pct_str"]].to_string(index=False))

▶ 총 이벤트 수: 15,460

=== 윈도우별 보존 결과 ===
        module  window  kept pct_str
classification       1 12202   78.9%
classification       2 12200   78.9%
classification       3 12199   78.9%
    regression       3 12199   78.9%
    regression       4 12199   78.9%
    regression       5 12198   78.9%
    regression       6 12197   78.9%
    regression       7 12196   78.9%
    regression       8 12195   78.9%
    regression       9 12194   78.9%
    regression      10 12188   78.8%
    clustering      11 12187   78.8%
    clustering      12 12182   78.8%
    clustering      13 12180   78.8%
    clustering      14 12177   78.8%
    clustering      15 12173   78.7%
    clustering      16 12156   78.6%
    clustering      17 11929   77.2%
    clustering      18 10913   70.6%
    clustering      19  8619   55.8%
    clustering      20  4254   27.5%
    clustering      21  1181    7.6%


In [13]:
# ── 03_features_common.ipynb 셀: 공통 피처(윈도우 無) 생성 ──

import os
import pandas as pd

# 1️⃣ 경로 설정
BASE        = "/Users/gun/Desktop/미래에셋 AI 공모전/data"
DIV_PATH    = os.path.join(BASE, "dividend_ml_ready.csv")
SECTOR_PATH = os.path.join(BASE, "sector_info.csv")
OUT_DIR     = os.path.join(BASE, "module_datasets")
os.makedirs(OUT_DIR, exist_ok=True)
OUT_PATH    = os.path.join(OUT_DIR, "features_common.csv")

# 2️⃣ 배당 이벤트 & 섹터 정보 로드
df = pd.read_csv(DIV_PATH, dtype={"stock_code":str, "rcept_no":str})
df["rcept_dt"] = pd.to_datetime(df["rcept_no"].str[:8], format="%Y%m%d", errors="coerce")
df = df.dropna(subset=["rcept_dt"]).reset_index(drop=True)

sec = pd.read_csv(SECTOR_PATH, dtype=str)
sec["stock_code"] = sec["stock_code"].str.zfill(6)

# 3️⃣ 같은 월(period) 내 per_share_common 순위 → div_amount_rank
df["period"] = df["rcept_dt"].dt.to_period("M")
df["div_amount_rank"] = (
    df.groupby("period")["per_share_common"]
      .rank(pct=True)
)

# 4️⃣ 공통 컬럼만 추출
common_cols = [
    "stock_code",
    "rcept_dt",
    "sector",
    "per_share_common",
    "yield_common",
    "total_amount",
    "div_amount_rank",
    "month",
    "is_year_end",
]
out = pd.DataFrame({
    "stock_code":       df["stock_code"],
    "rcept_dt":         df["rcept_dt"],
    "sector":           df["stock_code"].map(
                            sec.set_index("stock_code")["sector"]
                        ).fillna(""),  # 섹터 없으면 빈문자
    "per_share_common": df["per_share_common"],
    "yield_common":     df["yield_common"],
    "total_amount":     df["total_amount"],
    "div_amount_rank":  df["div_amount_rank"],
    "month":            df["rcept_dt"].dt.month,
    "is_year_end":      (df["rcept_dt"].dt.month == 12).astype(int),
})

# 5️⃣ 검증 및 저장
print(f"▶ 전체 이벤트 수: {len(df):,} → 공통 피처 보존: {len(out):,}")
print(out.isna().sum())
out.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print(f"✅ 공통 피처 저장 → {OUT_PATH}")

▶ 전체 이벤트 수: 15,460 → 공통 피처 보존: 15,460
stock_code          0
rcept_dt            0
sector              0
per_share_common    0
yield_common        0
total_amount        0
div_amount_rank     0
month               0
is_year_end         0
dtype: int64
✅ 공통 피처 저장 → /Users/gun/Desktop/미래에셋 AI 공모전/data/module_datasets/features_common.csv
